## Specifications

In [ ]:
import sys
from utils.setup import SetupPCAnalysis

argv           = sys.argv[1:]
# argv           = ['-c', 'cfg_pipeline.yml']

setup = SetupPCAnalysis(argv)


### Processing

In [ ]:
from utils.processing import proc_analysis

proc_analysis(
    setup.analysis,
    setup.gridpoints,
    setup.spcam_inputs,
    setup.spcam_outputs,
    setup.cond_ind_test,
    setup.ind_test_name,
    setup.pc_alphas,
    setup.levels,
    setup.parents_idx_levs,
    setup.children_idx_levs,
    setup.idx_lats,
    setup.idx_lons,
    setup.output_file_pattern,
    setup.output_folder,
    setup.overwrite_pc,
    setup.verbosity_pc
)